<a href="https://colab.research.google.com/github/pmontman/TSForecasting/blob/master/experiments/forecastingdata_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for accessing Forecastingdata forecasting and feature extraction tools using python

in Forecastingdata, some forecasting and feature calculation tools are implemented in R, but we can access them *transparently* from python using the awesome rpy2 package.
This notebook shows an example for it.


It has two parts. Part 1 is the preparation of the environment so everything
runs directly on python. Part2 is an example of how to use the tools directly from python.

## Part 1) Preparation of the environment

We load the R extension for jupyter, which interacts nicely with python

In [ ]:
#@title
%load_ext rpy2.ipython

Clone the repository, for reproducibility. No need to clone everytime if you have already have the folder!.

In [ ]:
!git clone https://github.com/pmontman/TSForecasting

In [ ]:
%cd TSForecasting/
!git pull
%cd ..

We now prepare the R environment for processing: installing and loading the required packages

```sh
cat << EOM > ~/.R/Makevars                                    
BREWDIR:=$(shell brew --prefix)
CPPFLAGS=-I$(BREWDIR)/include
LDFLAGS=-L$(BREWDIR)/lib
EOM

 brew install r harfbuzz fribidi
 brew install libgit2
 brew install freetype libpng libtiff zlib libjpeg-turbo
```


In [ ]:
%%R
install.packages("devtools")
library("devtools")
install.packages("Rcatch22")

In [ ]:
%%R
install.packages("tidyverse")
install.packages("tsibble")
install.packages("forecast")
install.packages("tsfeatures")
install.packages("smooth")

Install glmnet package for linear models

In [ ]:
%%R
install.packages('glmnet')

In [ ]:
! ls TSForecasting/experiments

In [ ]:
%%R
BASE_DIR <- "TSForecasting"
source("TSForecasting/experiments/feature_functions.R")
source("TSForecasting/experiments/fixed_horizon_functions.R")

The fixed horizon we need to try line by line to skip errors

In [ ]:
%%R
#source("TSForecasting/experiments/fixed_horizon_functions.R")

## Part 2) Using tools directly on Python

Load the function into python environment through rpy2 so
it can be later accessed directly from python code

In [ ]:
import rpy2.robjects as robjects
calculate_features = robjects.r['calculate_features']
do_fixed_horizon_local_forecasting = robjects.r['do_fixed_horizon_local_forecasting']
do_fixed_horizon_global_forecasting = robjects.r['do_fixed_horizon_global_forecasting']


we can now call `calculate_features` directly from python, here we have two examples using both the 'tsfeatures' and the 'catch22' features.
We will first remove the files just in case

Compute the features, we capture the stdout output to remove the verbosiness  of the progress report

In [ ]:
%%capture cap --no-stderr
calculate_features("bitcoin_without_missing", "bitcoin_dataset_without_missing_values.tsf", "series_name", "start_timestamp", "tsfeatures")
calculate_features("bitcoin_without_missing", "bitcoin_dataset_without_missing_values.tsf", "series_name", "start_timestamp", "catch22")

The functions write the outputs to .csv files in the project directory
structure. We can quickly check if the experiments runs OK, we should see some features in the following files.

In [ ]:
!echo "head of the tsfeatures features"
!ls -r TSForecasting/results/tsfeatures
!head -5 TSForecasting/results/tsfeatures/bitcoin_without_missing_features.csv
!echo "head of the catch22 features"
!ls -r TSForecasting/results/catch22_features
!head -5 TSForecasting/results/catch22_features/bitcoin_without_missing_features.csv

Compute the local forecasts.

In [ ]:
do_fixed_horizon_local_forecasting("bitcoin_without_missing", "ses", "bitcoin_dataset_without_missing_values.tsf", "series_name", "start_timestamp", 8)

Try the global model forecasts.

In [ ]:
do_fixed_horizon_global_forecasting("bitcoin_without_missing", 25, "bitcoin_dataset_without_missing_values.tsf", "pooled_regression", "series_name", "start_timestamp", 8)